In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

# Step 1

In [315]:
df = spark.read.format('csv').options(header='true', inferSchema='true').load('C:/_Emory/1_676-Machine Learning II/Assignments/HW2/train.csv')

# Step 2

In [316]:
# i) Print the dataset and verify that the schema contains all the variables
df.show()
df.count()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

891

In [317]:
# ii) Print the first 10 rows from the dataset. 
df.show(10)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [318]:
# iii) Obtain summary statistics for all variables in the dataframe. 
# Pay attention to whether there are missing data as well as whether the field appears to be continuous or discrete. 
df.describe().show()

from pyspark.sql.functions import isnull, when, count, col
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

df.dtypes

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

[('PassengerId', 'int'),
 ('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Ticket', 'string'),
 ('Fare', 'double'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

In [319]:
# iv) For each of the string columns (except name and ticket), print the count of the 10 most frequent values ordered by descending order of frequency.
from pyspark.sql.functions import col, countDistinct
df.groupby(df.Sex).count().distinct().sort('count',ascending=False).show()
#df.groupby(df.Ticket).count().distinct().sort('count',ascending=False).show(10)
df.groupby(df.Cabin).count().distinct().sort('count',ascending=False).show(10)
df.groupby(df.Embarked).count().distinct().sort('count',ascending=False).show()

#Useless code
#train.select("Sex").distinct().show()
#train.agg(*(countDistinct(col(c)).alias(c) for c in train.columns)).show()

+------+-----+
|   Sex|count|
+------+-----+
|  male|  577|
|female|  314|
+------+-----+

+-----------+-----+
|      Cabin|count|
+-----------+-----+
|       null|  687|
|C23 C25 C27|    4|
|         G6|    4|
|    B96 B98|    4|
|       E101|    3|
|    C22 C26|    3|
|        F33|    3|
|         F2|    3|
|          D|    3|
|        C83|    2|
+-----------+-----+
only showing top 10 rows

+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  644|
|       C|  168|
|       Q|   77|
|    null|    2|
+--------+-----+



In [ ]:
# v) Based on the above, which columns would you keep as features and which would you drop? Justify your answer. 
# drop cabin, because many missing values. 687/891
# drop Name and Ticket, because I believe this is unrelated with our prediction
# drop passenger_id, becasue this is just an row index
# keep all other features

# Step 3

In [320]:
# i) Select all feature columns you plan to use in addition to the target variable (i.e., ‘Survived’) and covert all numerical columns into double data type.
#Tip: you can use the .cast() from pyspark.sql.functions. 
df=df.drop('cabin')
df=df.drop('Name')
df=df.drop('Ticket')
df=df.drop('PassengerId')

df = df.withColumn("SibSp",df["SibSp"].cast("double"))
df = df.withColumn("Parch",df["Parch"].cast("double"))
df = df.withColumn("Pclass",df["Pclass"].cast("double"))

df.dtypes
df.count()

891

In [321]:
# ii) Replace the missing values in the Age column with the mean value. 
# Create also a new variable (e.g., ‘AgeNA’) indicating whether the value of age was missing or not. 
from pyspark.sql.functions import mean, avg
from pyspark.sql import functions as F
df=df.withColumn("AgeNA",
                       F.when((F.col("Age").isNull()), 1)\
                        .otherwise(0))
df=df.na.fill({'Age': 29.7})
df = df.withColumn("AgeNA",df["AgeNA"].cast("double"))
df.show()
df.count()

+--------+------+------+----+-----+-----+-------+--------+-----+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|AgeNA|
+--------+------+------+----+-----+-----+-------+--------+-----+
|       0|   3.0|  male|22.0|  1.0|  0.0|   7.25|       S|  0.0|
|       1|   1.0|female|38.0|  1.0|  0.0|71.2833|       C|  0.0|
|       1|   3.0|female|26.0|  0.0|  0.0|  7.925|       S|  0.0|
|       1|   1.0|female|35.0|  1.0|  0.0|   53.1|       S|  0.0|
|       0|   3.0|  male|35.0|  0.0|  0.0|   8.05|       S|  0.0|
|       0|   3.0|  male|29.7|  0.0|  0.0| 8.4583|       Q|  1.0|
|       0|   1.0|  male|54.0|  0.0|  0.0|51.8625|       S|  0.0|
|       0|   3.0|  male| 2.0|  3.0|  1.0| 21.075|       S|  0.0|
|       1|   3.0|female|27.0|  0.0|  2.0|11.1333|       S|  0.0|
|       1|   2.0|female|14.0|  1.0|  0.0|30.0708|       C|  0.0|
|       1|   3.0|female| 4.0|  1.0|  1.0|   16.7|       S|  0.0|
|       1|   1.0|female|58.0|  0.0|  0.0|  26.55|       S|  0.0|
|       0|   3.0|  male|2

891

In [322]:
from pyspark.sql.functions import isnull, when, count, col
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

+--------+------+---+---+-----+-----+----+--------+-----+
|Survived|Pclass|Sex|Age|SibSp|Parch|Fare|Embarked|AgeNA|
+--------+------+---+---+-----+-----+----+--------+-----+
|       0|     0|  0|  0|    0|    0|   0|       2|    0|
+--------+------+---+---+-----+-----+----+--------+-----+



In [323]:
# Deal with two null values in embarked, omit these two rows
df=df.dropna()
df.count()

889

In [324]:
# iii) Print the revised dataframe and recalculate the summary statistics. 
df.show()
df.describe().show()

+--------+------+------+----+-----+-----+-------+--------+-----+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|AgeNA|
+--------+------+------+----+-----+-----+-------+--------+-----+
|       0|   3.0|  male|22.0|  1.0|  0.0|   7.25|       S|  0.0|
|       1|   1.0|female|38.0|  1.0|  0.0|71.2833|       C|  0.0|
|       1|   3.0|female|26.0|  0.0|  0.0|  7.925|       S|  0.0|
|       1|   1.0|female|35.0|  1.0|  0.0|   53.1|       S|  0.0|
|       0|   3.0|  male|35.0|  0.0|  0.0|   8.05|       S|  0.0|
|       0|   3.0|  male|29.7|  0.0|  0.0| 8.4583|       Q|  1.0|
|       0|   1.0|  male|54.0|  0.0|  0.0|51.8625|       S|  0.0|
|       0|   3.0|  male| 2.0|  3.0|  1.0| 21.075|       S|  0.0|
|       1|   3.0|female|27.0|  0.0|  2.0|11.1333|       S|  0.0|
|       1|   2.0|female|14.0|  1.0|  0.0|30.0708|       C|  0.0|
|       1|   3.0|female| 4.0|  1.0|  1.0|   16.7|       S|  0.0|
|       1|   1.0|female|58.0|  0.0|  0.0|  26.55|       S|  0.0|
|       0|   3.0|  male|2

# Step 4

In [304]:
# i) Import all necessary pyspark functions.
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.feature import StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [325]:
# ii) Create indexers and encoders for categorical string variables. 
# Call them [field]_indexer and [field]_encoder, respectively. For instance, gender_indexer and gender_encoder. 

from pyspark.ml.feature import OneHotEncoder, StringIndexer

# StringIndexer
si_Sex = StringIndexer(inputCol="Sex",outputCol="Sex_indexer")
si_Embarked=StringIndexer(inputCol="Embarked",outputCol="Embarked_indexer")

# OneHotEncoder
oh_Sex = OneHotEncoder(inputCol="Sex_indexer",outputCol="Sex_encoder")
#oh_Pclass = OneHotEncoder(inputCol="Pclass",outputCol="Pclass_encoder")
#oh_SibSp = OneHotEncoder(inputCol="SibSp",outputCol="SibSp_encoder")
#oh_Parch = OneHotEncoder(inputCol="Parch",outputCol="Parch_encoder")
oh_Embarked=OneHotEncoder(inputCol="Embarked_indexer",outputCol="Embarked_encoder")


# Step 5

In [326]:
# The goal of this step is to assemble all feature columns into a feature vector in order to be used in the pipeline. 
# Tip: you can use the VectorAssembler to do this.
df.dtypes
from pyspark.ml.feature import VectorAssembler
# assembler = VectorAssembler(
#     inputCols=["Pclass_encoder","Sex_encoder","Age","SibSp_encoder","Parch_encoder","Fare","Embarked_encoder","AgeNA"],
#     outputCol="features")

assembler = VectorAssembler(
    inputCols=["Pclass","Sex_encoder","Age","SibSp","Parch","Fare","Embarked_encoder","AgeNA"],
    outputCol="features")

# Step 6

In [327]:
# create logistic regression model to be used in the pipeline
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="Survived",featuresCol="features")

# Step 7

In [328]:
# Assemble the pipeline
from pyspark.ml import Pipeline
#pl=Pipeline(stages=[si_Sex,si_Embarked,oh_Sex,oh_Pclass,oh_SibSp,oh_Parch,oh_Embarked,assembler,lr])
pl=Pipeline(stages=[si_Sex,si_Embarked,oh_Sex,oh_Embarked,assembler,lr])

# Step 8

In [329]:
# i) Use a 70-30 random split for the training and test sets, respectively. 
train, test = df.randomSplit([0.7, 0.3], seed = 29)

In [330]:
# ii) Verify the size of each dataset after the split.
print(train.count()/df.count(),test.count()/df.count())

0.7007874015748031 0.2992125984251969


# Step 9

In [331]:
# i) Fit the model using the predefined pipeline on the training set.
model=pl.fit(train)

In [332]:
# ii) Use the fitted model for prediction on the test set.
pred = model.transform(test)

In [333]:
# iii) Report the logistic regression coefficients. 
print(model.stages[-1].summary)

In [334]:
print(model.stages[-1].coefficients)
print(model.stages[-1].intercept)

[-1.0705783809790739,-2.5206792267576716,-0.03532335573370283,-0.29007011784877595,-0.08287229047753869,0.0008037718563865051,-0.5626823704920805,-0.13139324393926913,-0.5246659911084796]
5.213189202682474


# Step 10

In [335]:
# i) Print the first 5 rows of the results.
pred.show(5)

+--------+------+----+----+-----+-----+-------+--------+-----+-----------+----------------+-------------+----------------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass| Sex| Age|SibSp|Parch|   Fare|Embarked|AgeNA|Sex_indexer|Embarked_indexer|  Sex_encoder|Embarked_encoder|            features|       rawPrediction|         probability|prediction|
+--------+------+----+----+-----+-----+-------+--------+-----+-----------+----------------+-------------+----------------+--------------------+--------------------+--------------------+----------+
|       0|   1.0|male|18.0|  1.0|  0.0|  108.9|       C|  0.0|        0.0|             1.0|(1,[0],[1.0])|   (2,[1],[1.0])|[1.0,1.0,18.0,1.0...|[-0.6521785851115...|[0.34249876621813...|       1.0|
|       0|   1.0|male|19.0|  3.0|  2.0|  263.0|       S|  0.0|        0.0|             0.0|(1,[0],[1.0])|   (2,[0],[1.0])|[1.0,1.0,19.0,3.0...|[0.43645747075845...|[0.60741459397417...|       0.0|
|       0|   1.

In [337]:
pred.dtypes

[('Survived', 'int'),
 ('Pclass', 'double'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'double'),
 ('Parch', 'double'),
 ('Fare', 'double'),
 ('Embarked', 'string'),
 ('AgeNA', 'double'),
 ('Sex_indexer', 'double'),
 ('Embarked_indexer', 'double'),
 ('Sex_encoder', 'vector'),
 ('Embarked_encoder', 'vector'),
 ('features', 'vector'),
 ('rawPrediction', 'vector'),
 ('probability', 'vector'),
 ('prediction', 'double')]

In [336]:
# ii) Report the AUC for this model.
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction",labelCol='Survived')
evaluator.evaluate(pred)

0.8844339622641496